In [ ]:
import math
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import collections
from numpy import linalg
from collections import Counter
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/archive'
os.chdir(folder_path)
!ls

# Import data
df = pd.read_csv('articles1.csv')
#df2 = pd.read_csv('articles2.csv')
#df3 = pd.read_csv('articles3.csv')

# Merge data into one dataframe
#frames = [df1, df2, df3]
#data = pd.concat(frames)
#display(data.head())
# Convert data column into datetime
df['date'] = pd.to_datetime(df['date'])
columns_to_drop = ['Unnamed: 0', 'year', 'month', 'url']
df = df.drop(columns=columns_to_drop)
df = df.head(500)

Mounted at /content/drive


NameError: ignored

In [ ]:
def filter_common_words(df, threshold=0.95):
    # Count the frequency of each word in the dataframe
    word_counts = Counter()
    for idx, row in df.iterrows():
        words = row['content'].split()
        word_counts.update(words)

    # Determine the threshold for word frequency
    N = len(df)
    threshold = threshold * N

    # Filter out words that exceed the threshold
    common_words = []
    for word, count in word_counts.items():
      if count <= threshold:
        common_words.append(word)

    # Apply the filter function to each row of the "content" column
    def filter_row(row):
      words = row.split()
      filtered_words = []
      for word in words:
          if word in common_words:
              filtered_words.append(word)
      return ' '.join(filtered_words)

    df['content'] = df['content'].apply(filter_row)

    return df

df = filter_common_words(df)

In [ ]:
def remove_infrequent_words(df, min_frequency=5):
    # Combine all strings in the specified column into a single string
    text = ' '.join(df['content'])

    # Tokenize the combined string into words
    words = text.split()

    # Count the frequency of each word using Counter
    word_counts = Counter(words)

    # Identify words that occur less than min_frequency times
    infrequent_words = []
    for word, count in word_counts.items():
       if count < min_frequency:
        infrequent_words.append(word)

    # Function to remove infrequent words from a text
    def remove_infrequent(text):
        return ' '.join([word for word in text.split() if word not in infrequent_words])

    # Apply the remove_infrequent function to the specified column
    df['content'] = df['content'].apply(remove_infrequent)

    return df

In [ ]:
df = remove_infrequent_words(df)

In [ ]:
# Download NLTK resources
nltk.download('stopwords')

# Function to preprocess content column
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and digits
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in text.split() if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(stemmed_tokens)

# Apply preprocessing to the 'content' column
df['preprocessed_content'] = df['content'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Create a bag of words dictionary
bag_of_words = {}

# Iterate over the 'content' column
for content in df['preprocessed_content']:
    # Split the string into words
    words = content.split()

    # Update the word counts in the dictionary
    for word in words:
        if word in bag_of_words:
            bag_of_words[word] += 1
        else:
            bag_of_words[word] = 1

# Print the bag of words dictionary
print(bag_of_words)

{'washington': 388, 'congression': 86, 'republican': 588, 'fear': 86, 'come': 474, 'health': 381, 'care': 360, 'lawsuit': 68, 'obama': 614, 'might': 292, 'win': 134, 'incom': 116, 'administr': 318, 'choos': 40, 'longer': 87, 'defend': 90, 'execut': 321, 'branch': 28, 'suit': 47, 'challeng': 99, 'author': 216, 'spend': 119, 'billion': 154, 'dollar': 109, 'insur': 149, 'subsidi': 26, 'american': 153, 'hand': 180, 'hous': 589, 'big': 176, 'victori': 77, 'issu': 280, 'sudden': 15, 'loss': 62, 'disput': 39, 'caus': 137, 'program': 194, 'leav': 190, 'million': 418, 'without': 248, 'access': 92, 'prepar': 105, 'replac': 136, 'lead': 216, 'chao': 22, 'market': 183, 'spur': 13, 'polit': 485, 'backlash': 9, 'gain': 86, 'full': 96, 'control': 193, 'govern': 658, 'outcom': 24, 'find': 272, 'awkward': 10, 'posit': 170, 'huge': 44, 'temporarili': 5, 'law': 508, 'conserv': 94, 'voter': 61, 'demand': 107, 'end': 346, 'year': 982, 'anoth': 359, 'donald': 269, 'j': 320, 'worri': 65, 'fight': 156, 'alli'

In [ ]:
# Create vocabulary
def vocabulary(docs):
  vocab = set()
  for doc in docs:
    for word in doc:
        vocab.add(word)
  return sorted(vocab)

# Term frequency
def term_frequency(documents, vocabulary):
  tf_matrix = pd.DataFrame(0, index=np.arange(len(documents)), columns=vocabulary)
  for i, document in enumerate(documents):
    for word in document:
        tf_matrix.at[i, word] += 1
  return tf_matrix

# Inverse Document Frequency
def inverse_document_frequency(documents, vocabulary):
  idf = pd.Series(0, index=vocabulary)
  for word in vocabulary:
    counter = 0
    for document in documents:
      if word in document:
          counter += 1
    idf[word] = np.log((1+len(documents))/(counter+1))+1
  return idf

# TF-IDF
def tf_idf(tf_matrix, idf, documents):
  tfidf_matrix = tf_matrix.copy()
  for i in range(len(documents)):
    tfidf_matrix.iloc[i] = tf_matrix.iloc[i] * idf
  tfidf_matrix = normalize(tfidf_matrix, norm='l2', axis=1)
  tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=vocab)
  return tfidf_matrix

In [ ]:
# Extract documents
docs = df['preprocessed_content'].str.split()
# Vocabulary
vocab = vocabulary(docs)
# TF
tf_matrix = term_frequency(docs, vocab)
# IDF
idf = inverse_document_frequency(docs, vocab)
# TF-IDF
tfidf_matrix = tf_idf(tf_matrix, idf, docs)

In [ ]:
tfidf_matrix

,aaron,abandon,abc,abdul,abet,abil,abl,aboard,abort,abraham,...,youtub,youv,zavascki,zero,zervo,zhang,zilho,zink,zone,zubaydah
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.006520,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.009497,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.022921,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,0.0,0.000000,0.016760,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.0,0.0,0.0,0.0,0.0,0.000000,0.007244,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,0.0,0.0,0.0,0.0,0.0,0.000000,0.023988,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
